<a href="https://colab.research.google.com/github/SYWoo02/Engineer_BigData_Analysis_10/blob/main/Part2_%EC%9D%B4%EC%A7%84%EB%B6%84%EB%A5%98_%EC%97%B0%EC%8A%B5%EB%AC%B8%EC%A0%9C1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 이진분류 연습문제

## Section 1 환자의 당뇨병 여부 예측
### 환자의 당뇨병 여부를 예측하시오.
### 학습용 데이터(train)를 이용해 환자의 당뇨병을 예측하는 모델을 만든 후 이를 평가용 데이터(test)에 적용해 얻은 예측값을 다음과 같은 형식의 CSV 파일로 생성하시오.
* pred: 예측값(당뇨병일 확률)
* 제출 파일명: 'result.csv'
* 제출한 모델의 성능은 ROC-AUC 평가지표에 따라 채점한다.

In [21]:
# 라이브러리 및 데이터 불러오기
import pandas as pd

test = pd.read_csv("diabetes_test.csv")
train = pd.read_csv("diabetes_train.csv")

train
# Outcome이 당뇨병 발생여부 (0- 발생 X, 1-발생)

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,1,118,58,36,94,33.3,0.261,23,0
1,3,173,78,39,185,33.8,0.970,31,1
2,4,109,64,44,99,34.8,0.905,26,1
3,9,57,80,37,0,32.8,0.096,41,0
4,2,129,0,0,0,38.5,0.304,41,0
...,...,...,...,...,...,...,...,...,...
609,3,163,70,18,105,31.6,0.268,28,1
610,2,174,88,37,120,44.5,0.646,24,1
611,7,142,90,24,480,30.4,0.128,43,1
612,2,100,54,28,105,37.8,0.498,24,0


In [15]:
# 탐색적 데이터 분석 (EDA)
print("===== 데이터 크기 =====")
print("Train Shape:", train.shape)
print("Test Shape:", test.shape)
print("\n")

print("===== train 데이터 샘플 =====")
print(train.head(3))
print("\n")

print("===== test 데이터 샘플 =====")
print(test.head(3))
print("\n")

print("===== 데이터 정보(자료형) =====")
print(train.info())
print("\n")

print("===== train 결측치 수 =====")
print(train.isnull().sum())
print("\n")

print("===== test 결측치 수 =====")
print(test.isnull().sum())
print("\n")

print("===== traget 빈도 =====")
print(train['Outcome'].value_counts())
print("\n")

===== 데이터 크기 =====
Train Shape: (614, 9)
Test Shape: (154, 8)


===== train 데이터 샘플 =====
   Pregnancies  Glucose  BloodPressure  SkinThickness  Insulin   BMI  \
0            1      118             58             36       94  33.3   
1            3      173             78             39      185  33.8   
2            4      109             64             44       99  34.8   

   DiabetesPedigreeFunction  Age  Outcome  
0                     0.261   23        0  
1                     0.970   31        1  
2                     0.905   26        1  


===== test 데이터 샘플 =====
   Pregnancies  Glucose  BloodPressure  SkinThickness  Insulin   BMI  \
0            3      102             74              0        0  29.5   
1            5      104             74              0        0  28.8   
2            4       95             70             32        0  32.1   

   DiabetesPedigreeFunction  Age  
0                     0.121   32  
1                     0.153   48  
2                     0.61

In [22]:
# 데이터 전처리 (train에서 target인 Outcome만 뽑기)
target = train.pop('Outcome')

# train과 Validation 나누기 (7:3)
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(train, target, test_size=0.3, random_state=0)

# 머신러닝 학습 및 평가
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(random_state=0)
rf.fit(X_train, y_train)
pred = rf.predict_proba(X_val)

print("\n===== 예측 결과 확인 =====")
print(pred[:5])


===== 예측 결과 확인 =====
[[0.83 0.17]
 [0.22 0.78]
 [0.55 0.45]
 [0.75 0.25]
 [0.9  0.1 ]]


In [23]:
# 예측결과 평가 (Roc_AUC)
from sklearn.metrics import roc_auc_score
roc_auc = roc_auc_score(y_val, pred[:,1])
print("roc_auc:", roc_auc)

roc_auc: 0.806846116027532


In [24]:
# csv 파일 생성
pred = rf.predict_proba(test)
submit = pd.DataFrame({"pred":pred[:,1]})
submit.to_csv("result.csv",index=False)

# 생성된 파일 확인
result = pd.read_csv("result.csv")
result.head(5)

,pred
0,0.20
1,0.33
2,0.14
3,0.03
4,0.15
